# Limpieza de datos

La mayoría de la gente piensa que ser científico de datos significa estar corriendo modelos avanzados de Machine Learning todo el tiempo. La realidad es muy distinta:

![Tiempo de un científico de datos](./img/datascientist_time.jpeg)

La gran mayoría del tiempo se va **limpiando y organizando** los datos con los que queremos trabajar.

Los científicos de datos utilizamos herramientas como Pandas, Numpy, Matplotlib y Seaborn para limpiar los datos con los que queremos hacer algo.

## ETL

Un tipo de tarea que realizamos con gran frecuencia los científicos de datos son los **ETL**.

![Proceso de ETL](./img/etl.png)

ETL es un acrónimo que significa Extract, Transform, Load. Es un proceso que se utiliza para extraer datos de una fuente, transformarlos en un formato que sea adecuado para el análisis y cargarlos en una base de datos o algún otro sistema de almacenamiento.

## Ejemplo práctico

Los datos que usaremos para esta limpieza y nuestro siguiente análisis son datos de incidencia delictiva en nuestro país.

La iniciativa de datos abiertos del gobierno de México nos proporciona datos de incidencia delictiva desde 2015 hasta la fecha. Los datos se actualizan todos los meses y se pueden descargar desde el siguiente enlace: https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva

---

Como podemos ver en el portal, se proporcionan los datos tanto a nivel estatal como a nivel municipal. En este caso, utilizaremos los datos a nivel estatal.

Descarguemos los datos y guardemos el archivo CSV en la carpeta data con el nombre `datos_delitos.csv`.

---

Ahora leamos el archivo CSV y veamos cómo se ven los datos.

Primero que nada, importemos pandas

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/datos_delitos.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 1: invalid continuation byte

Aquí tenemos un error muy común que suele ocurrir cuando un archivo se guarda en una computadora con cierto "encoding". 

Un encoding es una tabla que relaciona un número con un carácter. Por ejemplo, en la tabla ASCII, el número 65 corresponde a la letra "A".

Si el archivo que estamos leyendo fue guardado con un encoding distinto al que pandas espera, nos arrojará un error. 

Para solucionar esto, podemos utilizar el parámetro `encoding` de la función `pd.read_csv()` y especificar el encoding correcto.

¿Pero cómo sabemos con qué encoding cuenta el archivo?

La realidad es que la gran mayoría de los archivos los encontrarán en encoding utf-8 y Pandas no va a dar ningún error. Aquí estamos teniendo este problema porque la computadora que utilizan para generar este archivo uso un encoding diferente a utf-8.

En México, por lo general, si un archivo no está en utf-8, lo más seguro es que esté en `ISO-8859-1` o `latin1`.

Intentemos cargar el archivo especificando el encoding `ISO-8859-1`.

In [ ]:
df = pd.read_csv('data/datos_delitos.csv', encoding='ISO-8859-1')
df.head()

Y vemos que ya podemos leer correctamente el archivo.


¿Existe alguna forma de verificar el encoding de un archivo sin tener que estar adivinando?

ChatGTP generó el siguiente código:

In [ ]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
    result = chardet.detect(rawdata)
    return result

file_path = './data/datos_delitos.csv'
encoding_info = detect_encoding(file_path)
print(f"Detected encoding: {encoding_info['encoding']}")

Sigamos...

In [3]:
df.tail(3)

NameError: name 'df' is not defined

In [4]:
df.info()

NameError: name 'df' is not defined

Bien. Están muy bien los datos. Sin embargo, tenemos un problema con el que es muy común encontrarnos. 

Resulta que el formato en el que está el archivo es fácil entender por seres humanos:


```markdown
| Año      | Entidad  | Enero    | Febrero  | Mes X    |
|----------|----------|----------|----------|----------|
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
```

Vemos que tenemos los meses como encabezados. Es decir, el archivo está tratando cada mes como si fuera una variable. 

Nosotros como científicos de datos estamos más interesados en conjuntos de datos que no estén en este formato de "resumen" o "tabla dinámica". Para nosotros, lo ideal sería que cada mes fuera simplemente una observación más en nuestro conjunto de datos. Es decir, queremos transformar la tabla de arriba en:

```markdown
| Año      | Entidad  | Mes        |
|----------|----------|------------|
|   ...    |   ...    |   Enero    |
|   ...    |   ...    |   Febrero  |
|   ...    |   ...    |   Marzo    |
|   ...    |   ...    |   Abril    |
|   ...    |   ...    |   Mes X    |
```

A este tipo de formato le llamos "formato largo de datos". 

Haremos las siguientes limpiezas:
* Transformar los nombres de las columnas para que no tengan caracteres especiales y estén siempre en minúsculas
* Convertir el dataset a un formato de datos "largo"

In [5]:
for col in df.columns:
    print(col)

NameError: name 'df' is not defined

In [6]:
len(df.columns)

NameError: name 'df' is not defined

### Limpiar nombres de columnas

In [7]:
def limpiar_columnas(df):
    columnas_limpias = []
    for col in df.columns:
        # convertir a minusculas, reemplazar espacios por guiones bajos y eliminar caracteres especiales
        col = col.lower().replace(" ", "_").replace("ñ", "ni").replace(".", "").replace("á", "a").replace("é", "e").replace("í","i").replace("ó", "o").replace("ú", "u")
        columnas_limpias.append(col)
    
    df.columns = columnas_limpias
    
    return df


In [8]:
df.head(2)

NameError: name 'df' is not defined

In [9]:
df = limpiar_columnas(df)

NameError: name 'df' is not defined

In [10]:
df.head(3)

NameError: name 'df' is not defined

Muy bien. Ahora lo que queremos hacer es quitar algunas columnas. Nos interesan nada más las siguientes:

In [11]:
df[['anio', 'entidad']]

NameError: name 'df' is not defined

In [12]:
df = df[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'subtipo_de_delito', 'modalidad','enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]
df.head(2)

NameError: name 'df' is not defined

### Formato largo de datos

Ahora, usaremos el método `melt` para convertir las columnas a observaciones.

Queremos convervar las coumnas:
* anio
* clave_ent
* entidad
* tipo_de_delito
* subtipo_de_delito
* modalidad

El resto de las columnas las vamos a juntar en una nueva columna llamada "nombre_mes" y sus valores los vamos a sumar en otra llamada "frecuencia"

In [13]:
print("Shape ", df.shape)

NameError: name 'df' is not defined

In [14]:
datos_long = df.melt(id_vars=['anio', 'clave_ent', 'entidad','tipo_de_delito', 'subtipo_de_delito', 'modalidad'], var_name='nombre_mes', value_name='frecuencia')

NameError: name 'df' is not defined

In [15]:
print("Shape: ", datos_long.shape)

NameError: name 'datos_long' is not defined

In [16]:
datos_long.head(5)

NameError: name 'datos_long' is not defined

In [17]:
datos_long.info()

NameError: name 'datos_long' is not defined

Supongamos que para este análisis, no nos importan los niveles subtipo de delito y modalidad. O sea, no queremos tener la distinción entre homicidios dolosos y culposos (sé que son bastante diferentes, pero simplifiquemos nuestro ejemplo).

Vamos a agrupar nuestro dataframe por anio, clave_ent, entidad, tipo_de_delito y nombre_mes. Esto hará que todos los tipos de homicidios se sumen al tipo "homicidio" o todos los tipos de robo de vehículo (con o sin violencia) se sumen a "robo de vehículo".

In [18]:
datos_long = datos_long.groupby(['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes'])['frecuencia'].sum().reset_index()

NameError: name 'datos_long' is not defined

In [19]:
datos_long[datos_long.tipo_de_delito == 'Robo'].sample(15)

NameError: name 'datos_long' is not defined

Mostremos todos los estados y su respectiva clave

In [20]:
datos_long[['clave_ent', 'entidad']].drop_duplicates().sort_values('entidad')

NameError: name 'datos_long' is not defined

Ahora Veamos todos los datos de delitos de una entidad en específico. Por ejemplo, Nuevo león

In [21]:
datos_long[datos_long['clave_ent'] == 19].sample(15)

NameError: name 'datos_long' is not defined

In [22]:
datos_long[(datos_long['clave_ent'] > 19) &  (datos_long['clave_ent'] < 24) & (datos_long['tipo_de_delito'] == 'Homicidio')].sample(15)

NameError: name 'datos_long' is not defined

### Valores de fechas

Finalmente, queremos tener una columna "fecha". Actualmente tenemos el año y el nombre del mes, pero no tenemos como tal una columna que tenga un tipo de dato fecha. Eso hace que filtrar por fecha sea complicado.

Por ejemplo, si queremos conocer todos los homicidios de Oaxaca en enero 2024, haríamos lo siguiente:

In [ ]:
datos_long[
    (datos_long['clave_ent'] == 20) &
    (datos_long['tipo_de_delito'] == 'Homicidio') &
    (datos_long['anio'] == 2024) &
    (datos_long['nombre_mes'] == 'enero') 
]

Creemos una columna de fecha. 

Primero tenemos que convertir el nombre de mes a un número, en donde 1 es enero, 2 febrero, etc.

In [24]:
datos_long.sample(2)

NameError: name 'datos_long' is not defined

In [25]:
datos_long["nueva_columna"] = "dato vacío"
datos_long.sample(6)

NameError: name 'datos_long' is not defined

In [26]:
# Diccionario de ayuda para convertir
meses = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

datos_long['mes'] = datos_long['nombre_mes'].map(meses)
datos_long.sample(5)

NameError: name 'datos_long' is not defined

In [27]:
datos_long["frecuencia_mas_10"] = datos_long["frecuencia"] + 10
datos_long.sample(5)

NameError: name 'datos_long' is not defined

In [28]:
datos_long["anio_mes"] = datos_long["anio"].astype(str) + datos_long["mes"].astype(str)
datos_long.sample(6)

NameError: name 'datos_long' is not defined

In [29]:
# yyyy-mm-dd, yy-mm-dd, yymmdd, yyyy/dd/mm

In [30]:
# Agregamos la columna de fecha juntando el año y el mes
datos_long['fecha'] = pd.to_datetime(datos_long['anio'].astype(str) + datos_long['mes'].astype(str), format='%Y%m')
datos_long.sample(5)

NameError: name 'datos_long' is not defined

In [31]:
# Eliminamos las columnas que ya no necesitamos
datos_long = datos_long.drop(columns=['nueva_columna'])
datos_long.sample(5)

NameError: name 'datos_long' is not defined

Veamos los homicidios en oaxaca de enero 2024 a la fecha

In [32]:
datos_long[
    (datos_long.tipo_de_delito == "Homicidio") &
    (datos_long.clave_ent == 20) &
    (datos_long.fecha >= '2024-01-01')
].sort_values('fecha')

NameError: name 'datos_long' is not defined

In [33]:
datos_finales = datos_long[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes', 'fecha', 'frecuencia']]
datos_finales.head(2)

NameError: name 'datos_long' is not defined

Ya que tenemos muestros datos bien estructurados, los podemos guardar en nuestra computadora. Los guardaremos con el nombre "delitos.csv"

In [34]:
datos_finales.to_csv('data/delitos.csv', index=False)

NameError: name 'datos_finales' is not defined